In [ ]:
import csv
import os
import gc
import numpy as np
import tensorflow as tf
from random import randint
from math import trunc
from tensorflow import keras
from keras.utils import np_utils, plot_model, to_categorical
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, concatenate

In [ ]:
# returns train, inference_encoder and inference_decoder models
def define_models(dim_o, dim_q, dim_p, n_units):
	# define training encoder
	enc_in_o = Input(shape=(None, dim_o))
	enc_in_q = Input(shape=(None, dim_q))
	enc_in_p = Input(shape=(None, dim_p))
	encoder_inputs = concatenate([enc_in_o, enc_in_q, enc_in_p])
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	
	# define training decoder
	dec_in_o = Input(shape=(None, dim_o))
	dec_in_q = Input(shape=(None, dim_q))
	dec_in_p = Input(shape=(None, dim_p))
	decoder_inputs = concatenate([dec_in_o, dec_in_q, dec_in_p])
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	dec_dense_o = Dense(dim_o, activation='softmax', name='tr_out_o')
	dec_dense_q = Dense(dim_q, activation='softmax', name='tr_out_q')
	dec_dense_p = Dense(dim_p, activation='softmax', name='tr_out_p')
	#out_o = Dense(1, activation='relu', name='tr_out_o')(decoder_outputs)#act relu
	#out_q = Dense(1, activation='sigmoid', name='tr_out_q')(decoder_outputs)
	#out_p = Dense(dim_p, activation='softmax', name='tr_out_p')(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)
	
	model = Model([enc_in_o, enc_in_q, enc_in_p, dec_in_o, dec_in_q, dec_in_p], [out_o, out_q, out_p])
	
	# define inference encoder
	encoder_model = Model([enc_in_o, enc_in_q, enc_in_p], encoder_states)
	
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	#out_o = TimeDistributed(Dense(1, activation='relu'))(decoder_outputs)#act relu
	#out_q = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)
	#out_p = TimeDistributed(Dense(dim_p, activation='softmax'))(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)

	decoder_model = Model([dec_in_o, dec_in_q, dec_in_p] + decoder_states_inputs, [out_o, out_q, out_p] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model


# generate target given source sequence
def predict_sequence(infenc, infdec, src_o, src_q, src_p, n_steps, cardinality_o, cardinality_q, cardinality_p):
	# encode
	state = infenc.predict([src_o, src_q, src_p])
	# start of sequence input
	target_o = np.array([0.0 for _ in range(cardinality_o)]).reshape(1, 1, cardinality_o)
	target_q = np.array([0.0 for _ in range(cardinality_q)]).reshape(1, 1, cardinality_q)
	#target_p = 0
	target_p = np.array([0.0 for _ in range(cardinality_p)]).reshape(1, 1, cardinality_p)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		#print(target_o.shape)
		#print(target_q.shape)
		#print(target_p.shape)
		#print(state[0].shape)
		o, q, p, h, c = infdec.predict([target_o, target_q, target_p] + state)
		#print(a)
		# store prediction
		output.append(o[0,0,:])
		output.append(q[0,0,:])
		output.append(p[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_o = o
		target_q = q
		target_p = p
	return np.array(output)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

In [ ]:
#load data
stream_list = []

with open('TRAAPPQ128F14961F5.txt', 'r') as f: 
    reader = csv.reader(f)
    sub_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]
    stream_list = stream_list + sub_list

In [ ]:
#create seperate data structures for each variable (offset, quarterlength, pitch)
#normalise offset and quarterlength
offs = []
qlngth = []
ptch = []

max_o = 600.0
max_q = 50.0

offsb = max(element[0] for element in stream_list if element[0]<=max_o)
qlngthb = max(element[1] for element in stream_list if element[1]<=max_q)
#ptchb = 127.0

for row in stream_list:
    if (row[0] <= max_o and row[1] <= max_q):
        offs.append(trunc(row[0]*100)/100)
        qlngth.append(trunc(row[1]*100)/100)
        ptch.append(row[2])

In [ ]:
fullql = []
for i in range(0, int(max_o), 1):
    #j = str(i)   
    fullql.append(i+.0) 
    fullql.append(i+.25)
    fullql.append(i+.33)
    fullql.append(i+.5)
    fullql.append(i+.66)
    fullql.append(i+.75)
fullql.append(max_o)
fullql.append(max_o+.25)
fullql[10] = 1.66
ql_to_int = dict((c, i) for i, c in enumerate(fullql))
int_to_ql = dict((i, c) for i, c in enumerate(fullql))

In [ ]:
del stream_list
gc.collect()

20

In [ ]:
#divide the sets in sequences of specific length 
dtlngth=len(offs)
n_features_o = int(max_o)*6+2
n_features_q = int(max_q)*6+2
n_features_p = 127+1
seq_length = 4#100 groups of 3

dataX1_o = []
dataX1_q = []
dataX1_p = []
dataX2_o = []
dataX2_q = []
dataX2_p = []

for i in range(0, dtlngth - seq_length + 1, 1):
	seq_in_o = offs[i:i + seq_length]
	seq_in_q = qlngth[i:i + seq_length]
	seq_in_p = ptch[i:i + seq_length]

	# create padded input target sequence
	target_in_o = [max_o+.25] + seq_in_o[:-1]
	target_in_q = [max_q+.25] + seq_in_q[:-1]
	target_in_p = [0] + seq_in_p[:-1]
	
	src_encoded_o = to_categorical([ql_to_int[ql] for ql in seq_in_o], num_classes=n_features_o)
	tar2_encoded_o = to_categorical([ql_to_int[ql] for ql in target_in_o], num_classes=n_features_o)
	src_encoded_q = to_categorical([ql_to_int[ql] for ql in seq_in_q], num_classes=n_features_q)
	tar2_encoded_q = to_categorical([ql_to_int[ql] for ql in target_in_q], num_classes=n_features_q)
	src_encoded_p = to_categorical(seq_in_p, num_classes=n_features_p)
	tar2_encoded_p = to_categorical(target_in_p, num_classes=n_features_p)
	
	dataX1_o.append(src_encoded_o)
	dataX1_q.append(src_encoded_q)
	dataX1_p.append(src_encoded_p)
	dataX2_o.append(tar2_encoded_o)
	dataX2_q.append(tar2_encoded_q)
	dataX2_p.append(tar2_encoded_p)

n_patterns = len(dataX1_p)
print ("Total Patterns: ", n_patterns)

Total Patterns:  7702


In [ ]:
del offs
del qlngth
del ptch
gc.collect()

In [ ]:
# configure problem
n_steps_out = seq_length
# define model
train, infenc, infdec = define_models(n_features_o, n_features_q, n_features_p, 256)
train.compile(optimizer='adam', loss={'tr_out_o': 'categorical_crossentropy', 'tr_out_q': 'categorical_crossentropy', 'tr_out_p': 'categorical_crossentropy'},
 metrics={'tr_out_o': 'accuracy', 'tr_out_q': 'accuracy', 'tr_out_p': 'accuracy'})

In [ ]:
#divide data in train and validation sets
dataX1_o_tr = dataX1_o[:-1000]
dataX1_o_v = dataX1_o[-1000:]
dataX2_o_tr = dataX2_o[:-1000]
dataX2_o_v = dataX2_o[-1000:]

dataX1_q_tr = dataX1_q[:-1000]
dataX1_q_v = dataX1_q[-1000:]
dataX2_q_tr = dataX2_q[:-1000]
dataX2_q_v = dataX2_q[-1000:]

dataX1_p_tr = dataX1_p[:-1000]
dataX1_p_v = dataX1_p[-1000:]
dataX2_p_tr = dataX2_p[:-1000]
dataX2_p_v = dataX2_p[-1000:]

In [ ]:
# train model
train.fit([np.array(dataX1_o), np.array(dataX1_q), np.array(dataX1_p), np.array(dataX2_o), np.array(dataX2_q), np.array(dataX2_p)], [np.array(dataX1_o), np.array(dataX1_q), np.array(dataX1_p)], epochs=10)

Epoch 1/10
241/241 [==============================] - 54s 222ms/step - loss: 11.4066 - tr_out_o_loss: 7.1613 - tr_out_q_loss: 1.5245 - tr_out_p_loss: 2.7207 - tr_out_o_accuracy: 0.0037 - tr_out_q_accuracy: 0.6298 - tr_out_p_accuracy: 0.2320
Epoch 2/10
241/241 [==============================] - 50s 206ms/step - loss: 7.4013 - tr_out_o_loss: 5.4633 - tr_out_q_loss: 0.6589 - tr_out_p_loss: 1.2792 - tr_out_o_accuracy: 0.0464 - tr_out_q_accuracy: 0.7732 - tr_out_p_accuracy: 0.5592
Epoch 3/10
241/241 [==============================] - 50s 208ms/step - loss: 4.8357 - tr_out_o_loss: 3.7865 - tr_out_q_loss: 0.3613 - tr_out_p_loss: 0.6879 - tr_out_o_accuracy: 0.2631 - tr_out_q_accuracy: 0.8813 - tr_out_p_accuracy: 0.7936
Epoch 4/10
241/241 [==============================] - 50s 208ms/step - loss: 2.8120 - tr_out_o_loss: 2.1374 - tr_out_q_loss: 0.2425 - tr_out_p_loss: 0.4320 - tr_out_o_accuracy: 0.6562 - tr_out_q_accuracy: 0.9230 - tr_out_p_accuracy: 0.8872
Epoch 5/10
241/241 [===================

In [ ]:
#int_to_ql[one_hot_decode(dataX1_o)[-1]]
#one_hot_decode(dataX1_o)[-1]
np.array(dataX1_p).shape

(32061, 4, 128)

In [ ]:
train.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 3602)] 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 302)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 3602)] 0                                            
_______________________________________________________________________________________

In [ ]:
# spot check some examples
for _ in range(10):
    i = randint(1, 1000)
    X1_o = np.reshape(dataX1_o[i], (1, seq_length, n_features_o))
    X1_q = np.reshape(dataX1_q[i], (1, seq_length, n_features_q))
    X1_p = np.reshape(dataX1_p[i], (1, seq_length, n_features_p))
    target = predict_sequence(infenc, infdec, X1_o, X1_q, X1_p, n_steps_out, n_features_o, n_features_q, n_features_p)
    for j in range(seq_length):
        print('X_o=%s, y_o=%s, X_q=%s, y_q=%s, X_p=%s, y_p=%s' % (
            int_to_ql[one_hot_decode([dataX1_o[i][j]])[0]], int_to_ql[one_hot_decode([target[3*j]])[0]], 
            int_to_ql[one_hot_decode([dataX1_q[i][j]])[0]], int_to_ql[one_hot_decode([target[3*j+1]])[0]], 
            one_hot_decode([dataX1_p[i][j]]), one_hot_decode([target[3*j+2]])))
    print()

X_o=38.0, y_o=38.0, X_q=0.25, y_q=0.25, X_p=[36], y_p=[36]
X_o=38.0, y_o=38.0, X_q=0.25, y_q=0.25, X_p=[48], y_p=[48]
X_o=38.0, y_o=38.0, X_q=0.5, y_q=0.5, X_p=[55], y_p=[55]
X_o=38.0, y_o=38.0, X_q=0.25, y_q=0.25, X_p=[24], y_p=[59]

X_o=26.5, y_o=26.5, X_q=0.5, y_q=0.5, X_p=[43], y_p=[43]
X_o=26.5, y_o=26.5, X_q=0.25, y_q=0.25, X_p=[43], y_p=[43]
X_o=26.5, y_o=26.5, X_q=0.25, y_q=0.25, X_p=[55], y_p=[55]
X_o=27.0, y_o=27.0, X_q=0.25, y_q=0.25, X_p=[43], y_p=[43]

X_o=11.5, y_o=11.5, X_q=0.25, y_q=0.25, X_p=[79], y_p=[79]
X_o=12.0, y_o=12.0, X_q=0.25, y_q=0.25, X_p=[41], y_p=[41]
X_o=12.0, y_o=12.0, X_q=0.25, y_q=0.25, X_p=[53], y_p=[53]
X_o=12.0, y_o=12.0, X_q=0.5, y_q=0.5, X_p=[41], y_p=[41]

X_o=19.5, y_o=19.5, X_q=0.25, y_q=0.25, X_p=[67], y_p=[67]
X_o=19.5, y_o=19.5, X_q=0.25, y_q=0.25, X_p=[59], y_p=[59]
X_o=19.5, y_o=19.5, X_q=0.25, y_q=0.25, X_p=[62], y_p=[62]
X_o=19.5, y_o=19.5, X_q=0.25, y_q=0.25, X_p=[67], y_p=[67]

X_o=44.5, y_o=44.5, X_q=0.25, y_q=0.25, X_p=[60], y_p=[60]

In [ ]:
int_to_ql[one_hot_decode([target[3*j]])[0]]

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def strided_axis0(a, fillval, L): # a is 1D array
    a_ext = np.concatenate(( np.full(L-1,fillval) ,a))
    n = a_ext.strides[0]
    strided = np.lib.stride_tricks.as_strided     
    return strided(a_ext, shape=(a.shape[0],L), strides=(n,n))

#if (rolling_window(np.asarray(offs), 6).all == np.asarray(dataX1_o).all):
#print(np.asarray(dataX1_o).shape)
#print(rolling_window(np.asarray(offs), 6).shape)
#print(dataX1_o[5460][2]*offsb)

ptch_fs = to_categorical(ptch, num_classes=127+1)
ptch_fs = rolling_window(np.asarray(ptch_fs), 6)
#ptch_fs = to_categorical(ptch_fs, num_classes=127+1)

#print (ptch_fs.shape)
#print (np.asarray(dataX1_p).shape)

#one_hot_decode(ptch_fs) == one_hot_decode(dataX1_p)
ptch_fs[-1]

#strided_axis0(np.asarray(ptch), 0, 6)